In [1]:
import pandas as pd
import sys, os
import numpy as np
import json
import shapefile

## Read shapefile

In [2]:
sf_path = os.path.join('new_link', 'new_link_latlon')
link_sf = shapefile.Reader(sf_path)
shapeRecs = link_sf.shapeRecords()

link_map = {}
for data in shapeRecs:
    link_map[data.record[1]] = data.shape.points
#     print(data.shape.points)
#     break

## Read link congestion file && process to geojson format

In [3]:
vehloc_path = os.path.join('..', '..', '..', 'data', 'input_files_MckeesRocks_SPC', 'veh_loc', 'veh_loc_raw.txt')
# final_str = 'date,longitude,latitude\n'
final_str = ''
with open(vehloc_path, 'r') as infile:
    lines = infile.readlines()
    for line in lines:
        data = line.rstrip().split(' ')
        time = int(data[1]) // 12
        if time < 360:
            delta_h = time // 60
            delta_m = time % 60
            hour = str(5 + delta_h)
            if len(hour) == 1:
                hour = '0' + hour
            if delta_m < 10:
                minute = '0' + str(delta_m)
            else:
                minute = str(delta_m)
            link_points = link_map[data[2]]
            porportion = float(data[3])
            loc = int(np.floor(porportion * len(link_points)))
            final_time = '2017-01-01T' + hour + ':' + minute + ':00Z'
            final_str += ','.join([str(e) for e in [time, round(link_points[loc][0],7), round(link_points[loc][1], 7)]]) + '\n'
#             print(final_str)
#             break

out_path = os.path.join('..', '..', '..', 'data', 'input_files_MckeesRocks_SPC', 'veh_loc', 'veh_loc_final.csv')
f = open(out_path, 'w')
f.write(final_str)
f.close()

3100209
[(-103.49651, 31.39598), (-103.49585, 31.39616), (-103.4952, 31.39638), (-103.49456, 31.39663), (-103.49393, 31.39688), (-103.4927, 31.39734)]
[-103.49651, 31.39598, -103.4927, 31.39734]
[0]
3


In [13]:
# create a new python dict to contain our geojson data, using geojson format
geojson = {'type':'FeatureCollection', 'features':[]}

df_by_tmc = df_valid.groupby('TMC')
for TMC, data in df_by_tmc:
    # create a feature template to fill in
    feature = {'type':'Feature',
               'properties':{},
               'geometry':{'type': 'MultiLineString',
                           'coordinates':[]
                }}
    
    for i in range(len(data['LINK_ID'])):
        _id = data['LINK_ID'].iloc[i]
        feature['geometry']['coordinates'].append(link_table[_id])
        
    # for each property get the value and add it as a new feature property
    e1 = df2[df2['TMC'] == TMC].iloc[0]
    feature['properties']['TMC'] = e1['TMC']
    feature['properties']['ROAD_NUMBER'] = e1['ROAD_NUMBER']
    feature['properties']['ROAD_NAME'] = e1['ROAD_NAME']
    feature['properties']['DISTANCE'] = round(e1['DISTANCE'], 2)
    feature['properties']['ROAD_DIRECTION'] = e1['ROAD_DIRECTION']
    
    am_peak_tt_str = ''
    am_peak_speed_str = ''    
    if speed_val[TMC]['AM_Peak_All'] != 0:
        am_peak_tt_str += str(round(speed_val[TMC]['AM_Peak_All'], 2)) + ' '
        am_peak_speed_str += str(round(e1['DISTANCE']/speed_val[TMC]['AM_Peak_All']*3600, 2)) + ' '
        feature['properties']['AM_SPEED_ALL'] = round(e1['DISTANCE']/speed_val[TMC]['AM_Peak_All']*3600, 2)
    else:
        am_peak_tt_str += ' N/A  '
        am_peak_speed_str += ' N/A  '
        feature['properties']['AM_SPEED_ALL'] = 0
    if speed_val[TMC]['AM_Peak_Car'] != 0:
        am_peak_tt_str += str(round(speed_val[TMC]['AM_Peak_Car'], 2)) + ' '
        am_peak_speed_str += str(round(e1['DISTANCE']/speed_val[TMC]['AM_Peak_Car']*3600, 2)) + ' '
        feature['properties']['AM_SPEED_CAR'] = round(e1['DISTANCE']/speed_val[TMC]['AM_Peak_Car']*3600, 2)
    else:
        am_peak_tt_str += ' N/A  '
        am_peak_speed_str += ' N/A  '
        feature['properties']['AM_SPEED_CAR'] = 0
    if speed_val[TMC]['AM_Peak_Truck'] != 0:
        am_peak_tt_str += str(round(speed_val[TMC]['AM_Peak_Truck'], 2)) + ' '
        am_peak_speed_str += str(round(e1['DISTANCE']/speed_val[TMC]['AM_Peak_Truck']*3600, 2)) + ' '
        feature['properties']['AM_SPEED_TRUCK'] = round(e1['DISTANCE']/speed_val[TMC]['AM_Peak_Truck']*3600, 2)
    else:
        am_peak_tt_str += ' N/A  '
        am_peak_speed_str += ' N/A  '
        feature['properties']['AM_SPEED_TRUCK'] = 0
        
    pm_peak_tt_str = ''
    pm_peak_speed_str = ''    
    if speed_val[TMC]['PM_Peak_All'] != 0:
        pm_peak_tt_str += str(round(speed_val[TMC]['PM_Peak_All'], 2)) + ' '
        pm_peak_speed_str += str(round(e1['DISTANCE']/speed_val[TMC]['PM_Peak_All']*3600, 2)) + ' '
        feature['properties']['PM_SPEED_ALL'] = round(e1['DISTANCE']/speed_val[TMC]['PM_Peak_All']*3600, 2)
    else:
        pm_peak_tt_str += ' N/A  '
        pm_peak_speed_str += ' N/A  '
        feature['properties']['PM_SPEED_ALL'] = 0
    if speed_val[TMC]['PM_Peak_Car'] != 0:
        pm_peak_tt_str += str(round(speed_val[TMC]['PM_Peak_Car'], 2)) + ' '
        pm_peak_speed_str += str(round(e1['DISTANCE']/speed_val[TMC]['PM_Peak_Car']*3600, 2)) + ' '
        feature['properties']['PM_SPEED_CAR'] = round(e1['DISTANCE']/speed_val[TMC]['PM_Peak_Car']*3600, 2)
    else:
        pm_peak_tt_str += ' N/A  '
        pm_peak_speed_str += ' N/A  '
        feature['properties']['PM_SPEED_CAR'] = 0
    if speed_val[TMC]['PM_Peak_Truck'] != 0:
        pm_peak_tt_str += str(round(speed_val[TMC]['PM_Peak_Truck'], 2)) + ' '
        pm_peak_speed_str += str(round(e1['DISTANCE']/speed_val[TMC]['PM_Peak_Truck']*3600, 2)) + ' '
        feature['properties']['PM_SPEED_TRUCK'] = round(e1['DISTANCE']/speed_val[TMC]['PM_Peak_Truck']*3600, 2)
    else:
        pm_peak_tt_str += ' N/A  '
        pm_peak_speed_str += ' N/A  '
        feature['properties']['PM_SPEED_TRUCK'] = 0
    
    feature['properties']['AM_PEAK_TT'] = am_peak_tt_str
    feature['properties']['AM_PEAK_SPEED'] = am_peak_speed_str
    feature['properties']['PM_PEAK_TT'] = pm_peak_tt_str
    feature['properties']['PM_PEAK_SPEED'] = pm_peak_speed_str
    
    # add this feature to the list of features inside our dict
    geojson['features'].append(feature)
    
geojson_str = json.dumps(geojson, indent=2)
# save the geojson result to a file
output_filename = 'Speed_from_HERE_alldata.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var data_speed = {};'.format(geojson_str))

# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson['features'])))

1916 geotagged features saved to file
